## [복습]


### 1. iframe

 iframe 태그로 구성되어 있는 html 페이지의 경우 request를 iframe의 소스 url로 보내야 한다.
 
 
### 2. selenium

 브라우저의 동작에 따라 element를 찾을 수도 있고, 못 찾을 수도 있다. element 선택은 xpath, class, id, css선택자 등으로 모두 가능하다. 단수 선택, 복수 선택 모두 가능하다.
 드라이버가 올바른 데이터를 받아올 때까지 기다렸다가 페이지 소스를 받아와야 한다.
 
---
 
## [이전 시간 질문]

1. 동적 handling : 과거의 댓글까지 모두 가져오려고 했는데, 닉네임마다 동적으로 정보가 변한다.
> 동적으로 뜨는 댓글. F12 개발자도구 네트워크에서 더보기를 누를 때 어떻게 응답이 오는지 보면 된다. 응답으로 오는 메시지를 분석해서 핸들링한다. 즉, request를 똑같이 날려야 한다. 헤더 부분을 참조해서 request url 찾고, 각 부분에 맞춰서 넣어주면 된다.

# ---------------------------- 0604 수업 ----------------------------

## 인사이트

- 파이널 프로젝트 진행하면서 그 사람이 썼던 댓글 필터링해야 하는지 아닌지에 대해 궁금증이 있었다. 오늘처럼 그 사람의 뉴스 히스토리를 모을 수 있다면 분석하는 데에 도움이 될 것이다.

- 이전 파이널 프로젝트 때 댓글 크롤링할 때는 selenium 이용해서 더보기 버튼을 계속해서 클릭해 줬다. 그것보다 지금처럼 요청을 한 번에 보내고 json 데이터로 읽어와서 저장하는 게 더 빠를 것으로 보인다.


## 제발...

* `parse_qs`의 키, 값 쌍에서 값은 리스트다. 그대로 넣으면 안 된다. 인덱싱 통해 리스트 안의 문자열에 접근해야 한다!
* request 보낼 때 안 되면 제일 먼저 url부터 확인합시다...


## 추후 고민
- `sid` 파라미터
- `userId` : user key값. --> 나중에 이거 바꾸면 그 사람의 히스토리 모두 다 가져올 수 있을 듯?
- `page`, `pageSize` 활용하지 말고, 댓글 개수 읽어서 해당하는 개수만큼 json 데이터 가져오도록 조정하는 것은?

# Dynamic DOM 크롤링

## 크롤링 목표
1. 네이버 뉴스 리스트를 가져온다.
2. 네이버 뉴스 리스트에 올라 있는 뉴스를 추출한다.
3. 해당 네이버 뉴스의 댓글들을 추출한다.
4. 댓글을 작성한 사용자에 기반해, 그 사용자가 작성한 댓글들을 모두 추출한다.

## 크롤링 개발 순서도

1. URL 분석
    - 1) XML / JSON / iframe 여부
    - 2) 1)로 request 전송했을 때, 예상하던 response를 받을 수 있는지 여부.
    - 3) 2)가 False일 경우, request 다시 확인.
        - **`header`**와 **`params`**가 맞는지.
        - 아니라면 다른 방법 모색 ex)Selenium.
    
    
2. Parsing
    - 1) 데이터 형태 : XML / JSON
    - 2) 불순한 데이터가 존재하는지 여부 확인. 만약 존재한다면, 문자열 함수 등을 이용해 제거.
    - 3) 각 데이터 형태에 맞는 방식으로 파싱.
        - XML : `BeautifulSoup` 라이브러리 이용.
        - JSON : `json.loads` 혹은 `resp.json()` 이용.

In [85]:
# module import
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import json
from urllib.parse import parse_qs, urlparse

# 1. 네이버 뉴스 URL을 분석한다.


> 모든 페이지의 기사 목록을 가져 온다. 이미지까지 포함해서 가져 온다.


### XML : BeautifulSoup 사용



1. 가져와야 하는 정보 및 사이트
    - 네이버 경제, 금융 뉴스
    - 사이트 : https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259

---

2. URL 분석(url 파라미터)
    - 카테고리 : `sid1`, `sid2`
        - 경제(큰 카테고리) : `sid1`
        - 하위 카테고리
            - 금융 : https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259
            - 증권 : https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=258
    - 날짜 : date



In [156]:
# url 분해
base_url = "https://news.naver.com/main/list.nhn" # 뒷부분 : ?mode=LS2D&mid=shm&sid1=101&sid2=259
params = {
    'mode' : 'LS2D',
    'mid' : 'shm',
    'sid1' : 101, # category1 : 경제
    'sid2' : 259, # category2 : 금융
    'date' : '20200602' # date
}

# 요청 보내고 제대로 오는지 체크
req = requests.get(base_url, params=params)
print(f"URL: {req.url}")
print(f"Status Code: {req.status_code}")

# 텍스트 데이터 잘 수신했는지 확인
# print(req.text)

URL: https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&date=20200602
Status Code: 200


# 2. 네이버 뉴스 내용을 가져 온다.


### XML : BeautifulSoup 사용



**구조 확인**
    
![news-cat](./images/web_21.png)

    - 태그 구조 확인 : `ul` > `li`
    - 작업 개략 : 반복문 돌려서 li 태그만 다 찾아온다.
    
    
    
**내용 추출 : 헤드라인 + 상세 내용**
    
![news](./images/web_22.png)
    
    
    * 이미지 있는 경우
        - 첫 번째 `dt` : 이미지
        - 두 번째 `dt` : 헤드라인
    * 이미지 없는 경우 


In [157]:
# soup객체 만들기
soup = BeautifulSoup(req.text)

# 반복 통해 뉴스 헤드라인 추출
news_headline_wrap_uls = soup.find('ul', class_='type06_headline')
news_wrap_uls = soup.find('ul', class_='type06')
news_headline_list = news_headline_wrap_uls.find_all('li', recursive=False) + news_wrap_uls.find_all('li', recursive=False) # li를 하나씩만 가져오면 되므로, recursive 옵션 False.
print(f"Length of Headline List : {len(news_headline_list)}")

Length of Headline List : 20


In [158]:
# 사진이 있는 경우와 없는 경우 분리
news_results = []

for news_headline in news_headline_list:    
    
    # dt 태그 
    dt_tags = news_headline.find_all('dt')    
    if len(dt_tags) > 1: # 이미지 있음.
        img_src = dt_tags[0].find('img')['src'] # 이미지 url
        headline = dt_tags[1].find('a').get_text(strip=True)
        news_url = dt_tags[1].find('a')['href']        
    elif len(dt_tags) == 1: # 이미지 없음.
        img_src = None 
        headline = dt_tags[0].find('a').get_text(strip=True)
        news_url = dt_tags[0].find('a')['href']        
    else: # dt가 없는 경우 : 모르겠음.
        continue
    
    
    # dd 태그 : description
    dd_tags = news_headline.find('dd')    
    if dd_tags: # dd 태그 있음.
        desc = dd_tags.find('span', class_='lede').get_text(strip=True)
        writing = dd_tags.find('span', class_='writing').get_text(strip=True)        

    else:
        desc = None
        writing = None
        
    # 정보를 모아서 news_dict 생성  
    news_dict = {
        'image' : img_src,
        'headline' : headline,
        'url' : news_url,
        'desc' : desc,
        'press' : writing        
    }
    
    # 결과 리스트에 저장
    news_results.append(news_dict)

# 3. 네이버 뉴스의 댓글을 가져 온다.

### 순서도

* 문서 구조 파악.
* request 찾기.
* url 분석.
* parameter 변환.
* response 분석.
* response parsing
* 저장

### 코드 순서 : 요청하기 -> 분석하기


> 뉴스 상세 페이지에 들어가서 댓글을 가져온다.

- 댓글 더보기 탭을 찾아야 한다.
- 댓글이 많으면 댓글 더보기 페이지에서도, 더 보기 버튼을 계속 눌러야 한다.
- 이 요청을 개발자도구 네트워크 탭을 통해 확인하면 다음과 같다.
    
![comments](./images/web_23.png)


**맨 아래 셀에서 진행한 연습을 바탕으로 함수 만들어 받아온다.**

* `request_comment_list`
    - 한 뉴스 기사에 대한 모든 댓글의 json 데이터를 text 형태로 받아 온다.
    - 이 함수에서는 일단 요청만 보낸다.
    - 따라서 json 댓글 데이터가 문자열로 되어 있는 html 페이지가 반환될 것이다.
    
    
* `request_user_comment_list`
    - 해당 유저의 댓글 히스토리. 
    - comment number만 파라미터로 추가된다.
    
    
* `comment_resp_to_json` 
    - 댓글들의 리스트에 요청을 보내 받아 온 결과 html을 text로 반환한다.
    - json으로 바꾼다.
    - 파이썬의 딕셔너리 형태로 반환한다.

In [217]:
def request_comment_list(oid, aid):
    
    """
    뉴스 url을 기반으로 해당 뉴스 댓글 데이터 페이지에 request를 보내는 함수.
    
    * parameter
        - oid: 본문 url의 oid 쿼리.
        - aid: 본문 url의 aid 쿼리.
    
    * return
        - 요청을 보낸 후 파싱한 html.
    
    """
    
    # 댓글 api url
    comment_base_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json"
    
    # 댓글 페이지 comment parameter
    params = {
        'ticket' : 'news',
        'pool' : 'cbox5',
        'lang' : 'ko',
        'country': 'KR',       
        
        'objectId' : f'news{oid},{aid}', # 각각의 뉴스 url --> 바꿔야할 값.
        'pageSize' : 1000, # --> 그 이상이면 바꿔서 요청 한 번 더 보내야 함.
        'indexSize': 10,
        'page': 1, # --> 페이지
        'includeAllStatus': 'true',
        'cleanbotGrade':2
    }
    
    # 댓글 페이지 요청 headers
    headers = {
        'referer': f"https://news.naver.com/main/read.nhn?m_view=1&includeAllCount=true&mode=LS2D&mid=shm&sid1=101&sid2=259&oid={oid}&aid={aid}"
    }
    
    # URL 요청 보내기
    req = requests.get(comment_base_url, params=params, headers=headers)
    # print(f"해당 뉴스에서의 댓글 URL: {req.url}  / status: {req.status_code}")
    
    html = req.text

    return html

In [227]:
def request_user_comment_list(oid, aid, comment_no):
    
    """
    뉴스 url에서 접속한 해당 뉴스 댓글 데이터 페이지를 이용해,
    각 사용자가 작성한 댓글 데이터 페이지에 request를 보내는 함수.
    
    * parameter
        - oid: 뉴스 본문 url의 oid 쿼리.
        - aid: 뉴스 본문 url의 aid 쿼리.
        - comment_no: 해당 사용자를 식별하는 commentNo 쿼리.
    
    * return
        - 요청을 보낸 후 파싱한 html.
    """
    
    # 댓글 api url
    user_comment_base_url = "https://apis.naver.com/commentBox/cbox/web_neo_user_info_jsonp.json"
    
    # 댓글 페이지 comment parameter
    params = {
        'ticket' : 'news',
        'pool' : 'cbox5',
        'lang' : 'ko',
        'country': 'KR',     
        'objectId' : f'news{oid},{aid}', # 각각의 뉴스 url --> 바꿔야할 값.
        'pageSize' : 1000, # --> 그 이상이면 바꿔서 요청 한 번 더 보내야 함.
        'indexSize': 10,
        'page': 1, # --> 페이지
        'includeAllStatus': 'true',
        'cleanbotGrade':2,
        'commentNo' : comment_no
    }
    
    # 댓글 페이지 요청 headers
    headers = {
        'referer': f"https://news.naver.com/main/read.nhn?m_view=1&includeAllCount=true&mode=LS2D&mid=shm&sid1=101&sid2=259&oid={oid}&aid={aid}"
    }
    
    # URL 확인
    req = requests.get(user_comment_base_url, params=params, headers=headers)
    # print(f"해당 유저가 작성한 댓글의 URL: {req.url}  / status: {req.status_code}")
    
    html = req.text

    return html

In [219]:
def comment_resp_to_json(html):
    
    """
    응답으로 온 html 텍스트를 분석하여 파이썬의 객체로 만드는 함수.
    
    * parameter
        - html : <Response.text>
    
    * return
        - 응답 데이터가 저장되어 있는 파이썬 dict.
    
    """
    
    comment_text = html[10:-2] # comment_text : 문자열.
    comment_resp_dict = json.loads(comment_text) # json 데이터를 파이썬의 dictionary 형태로 바꿔 준다.
    
    # 댓글 몇 개 있는지 체크
    comment_list = comment_resp_dict['result']['commentList']
    print(f"Total: {len(comment_list)}")

    return comment_resp_dict

### 댓글 저장

 댓글 json api를 저장한다. 나중에 필요한 부분만 따다 쓰면 됨.
 
### 해당 유저의 댓글 히스토리 저장

> 해당 유저가 작성한 댓글의 히스토리에 접근하는 방법을 아직 강사님은 발견하지 못하셨다고 합니다! 일단 어떤 뉴스 들어가서 그 사람의 id를 통해 접근해서 히스토리를 모으는 방법을 사용한다.

* 위에서 저장한 댓글을 더 보기하면, commentNo가 뜬다.
* commentNo로 url 바꿔주면 그 사람의 댓글 히스토리를 저장할 수 있다.

In [242]:
user_comment_history = {}

for result in news_results:
    
    url = result['url'] # 해당 뉴스 주소
    parsed_url = urlparse(url) # 쿼리셋
    queries = parse_qs(parsed_url.query)
    oid, aid = queries['oid'][0], queries['aid'][0] # oid, aid 바뀐다!
    print(f"뉴스 페이지 주소: {url}")
    
    # 댓글 페이지에 요청 보내기
    comment_html = request_comment_list(oid, aid)
    
    # html -> json -> dict
    comment_dict = comment_resp_to_json(comment_html)
    comment = comment_dict.get('result').get('commentList')
    
    result['comment'] = comment
    
    # 각 댓글의 작성자에게 요청 보내기
    if comment is None:
        continue
    
    for c in comment:
        commentNo = c.get('commentNo')
        user_comment_html = request_user_comment_list(oid, aid, commentNo)
        
        # html -> json -> dict
        user_comment_dict = comment_resp_to_json(user_comment_html)
        user_comment = user_comment_dict.get('result').get('commentList')
        # pprint(user_comment)
        
        user_id = user_comment[0].get('userIdNo') # 어차피 안에 있는 댓글들의 작성자는 다 똑같다.
        user_name = user_comment[0].get('userName')
        
        # 키 설정
        user_key = f"{user_id} / {user_name}"
        
        user_comment_history.setdefault(user_key, [])
        user_comment_history[user_key].append(user_comment)

뉴스 페이지 주소: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=022&aid=0003471097
Total: 2
Total: 80
Total: 58
뉴스 페이지 주소: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=028&aid=0002499591
Total: 5
Total: 100
Total: 100
Total: 100
Total: 100
Total: 100
뉴스 페이지 주소: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=421&aid=0004673081
Total: 1
Total: 100
뉴스 페이지 주소: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=421&aid=0004673055
Total: 0
뉴스 페이지 주소: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=005&aid=0001327728
Total: 0
뉴스 페이지 주소: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=016&aid=0001681395
Total: 0
뉴스 페이지 주소: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=018&aid=0004655069
Total: 0
뉴스 페이지 주소: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=101&sid2=259&oid=029&aid=000260

# 댓글 json으로 받아오기 연습

### 요청 보내주기

1. parameter 살펴보기

    - parameter가 많다.
    - parameter 주석처리해보면서 하나씩 바꿔 보고, 제대로 받아오는지 확인한다.
    - 나중에 필요할 수 있지만 지금 당장 필요한 것은 아닐 수 있다.
    - 공백으로 되어 있는 건 일단 주석처리해도 된다.


---
### 데이터 받아 오기

2. 댓글 json 데이터로 받아 온다.
    - javascript 코드 문자열 바꿀 수 있다.
    - `json.load` 진행해야 함.
    - 문자열 슬라이싱 통해 잘라줌.
        ```
        '_callback({"success":true,"code":"1000","message":"요청을 성공적으로 처리하였습니다.","lang":"ko","country":"KR","result":{"commentList":[{"ticket":"news"
        
        (중략...)
        
        "sort":"FAVORITE","bestList":[]},"date":"2020-06-04T01:34:00+0000"});'
        ```
---
### 한 번에 진행
3. 변수 조정을 통해 한 번에 받아올 수 있지 않을까? + 파라미터 정리하자.
    - pageSize
    - page
    - objectId
       

In [ ]:
comment_resp = requests.get(comment_base_url, params=params, headers=headers)
print(comment_resp.url)

# 확인
comment_text = comment_resp.text
comment_text = comment_text[10:-2] # comment_text : 문자열.
comment_resp_dict = json.loads(comment_text) # json 데이터를 파이썬의 dictionary 형태로 바꿔 준다.
comment_list = comment_resp_dict['result']['commentList']
print(f"Total Comments: {len(comment_list)}")
print(comment_list)

### 댓글 더 보기 창에 들어가기 위해 어떤 요청을 보내야 하는가?

> 요청을 보낼 때 네트워크에서 웹이 어떻게 동작하는지 찾는다. request와 response의 내용을 분석한 후, 바뀌는 파라미터만 찾아서 요청을 다르게 보내주면 된다.

![header](./images/web_24.png)

![url](./images/web_25.png)


---


- requests 보낼 때 바뀌는 url 분석.
- 파라미터 일일이 분석 후, 동적으로 컨트롤해야 할 요소 필터링하기 : `pageSize`, `page`, `objectId`

    

In [66]:
# 1. 댓글 요청 url 분석

comment_base_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json"

# ?ticket=news&templateId=default_economy&pool=cbox5&_callback=jQuery1124021036404387583252_1591233733835
# &lang=ko&country=KR&objectId=news015%2C0004352610&categoryId=&pageSize=20
# &indexSize=10&groupId=&listType=OBJECT
# &pageType=more&page=3&refresh=false&sort=FAVORITE&current=2142689022
# &prev=2142644342&includeAllStatus=true&cleanbotGrade=2&_=1591233330121

params = {
    'ticket': 'news',
    'templateId': 'default_economy',
    'pool':'cbox5',
    # '_callback': '',
    'lang':'ko',
    'country': 'KR',
    'objectId':'news015,0004352610',
    # 'categoryId': '',
    'pageSize': 20,
    'indexSize': 10,
    # 'groupId':'',
    'listType':'OBJECT',
    'pageType': 'more',
    'page': 2,
    'refresh': 'false',
    'sort': 'FAVORITE',    
    # 'current': 2142689022,
    # 'prev': 2142644342,
    'includeAllStatus': 'true',
    'cleanbotGrade':2,
    # '_': 1591233330121
}

# 헤더 맞춰주기 :referer
headers= {
    'referer': 'https://news.naver.com/main/read.nhn?m_view=1&includeAllCount=true&mode=LS2D&mid=shm&sid1=101&sid2=259&oid=015&aid=0004352610'}
comment_resp = requests.get(comment_base_url, params=params, headers=headers)
# comment_resp.text

In [70]:
# 2. json 데이터 분석

comment_text = comment_resp.text
comment_text = comment_text[10:-2] # comment_text : 문자열.
comment_resp_dict = json.loads(comment_text) # json 데이터를 파이썬의 dictionary 형태로 바꿔 준다.
print(type(comment_resp_dict))
print('')
# pprint(comment_resp_dict) # 파이썬의 json 객체가 된다.

# 댓글 확인
comment_list = comment_resp_dict['result']['commentList']
print(f"Total Comments: {len(comment_list)}")

<class 'dict'>

Total Comments: 20


In [214]:
# 3. url 찾기 연습 : pageSize, page 조정을 통해 한 번에 가져오자. + 파라미터 정리.
comment_base_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json"

params = {
    'ticket': 'news',
    # 'templateId': 'default_economy',
    'pool':'cbox5',
    # '_callback': '',
    'lang':'ko',
    'country': 'KR',
    'objectId':'news015,0004352610', # --> 이 부분 oid, aid로 조정하면 된다.
    # 'categoryId': '',
    'pageSize': 100, # -> 이 부분 조정.
    'indexSize': 10,
    # 'groupId':'',
    #'listType':'OBJECT',
    #'pageType': 'more',
    'page': 1, # -> 이 부분 조정.
    #'refresh': 'false',
    #'sort': 'FAVORITE',    
    # 'current': 2142689022,
    # 'prev': 2142644342,
    'includeAllStatus': 'true',
    'cleanbotGrade':2,
    # '_': 1591233330121
}

# 헤더 맞춰주기 :referer
headers= {
    'referer': 'https://news.naver.com/main/read.nhn?m_view=1&includeAllCount=true&mode=LS2D&mid=shm&sid1=101&sid2=259&oid=015&aid=0004352610'}
comment_resp = requests.get(comment_base_url, params=params, headers=headers)
print(comment_resp.url)

# 확인
comment_text = comment_resp.text
comment_text = comment_text[10:-2] # comment_text : 문자열.
comment_resp_dict = json.loads(comment_text) # json 데이터를 파이썬의 dictionary 형태로 바꿔 준다.
comment_list = comment_resp_dict['result']['commentList']
print(f"Total Comments: {len(comment_list)}")

https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&pool=cbox5&lang=ko&country=KR&objectId=news015%2C0004352610&pageSize=100&indexSize=10&page=1&includeAllStatus=true&cleanbotGrade=2
Total Comments: 56


### 최종 정리 : 뉴스 url 바꿔 가면서 댓글 json 가져오기

* 축약해야 할 파라미터를 찾는다.
* 각각의 뉴스 url에서의 `oid`와 `aid` 부분이 json 데이터에서 `ObjectID`를 이룬다.

```
comment_base_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json"

params = {
    'ticket' : 'news',
    'pool' : 'cbox5',
    'lang' : 'ko',
    'country': 'KR',
    'objectId' : 'news015,0004352610', # --> 바꿔야할 값.
    'pageSize' : 1000, # 그 이상이면 바꿔서 요청 한 번 더 보내야 함.
    'page': 1, # 페이지
    'includeAllStatus': 'true',
    'cleanbotGrade':2
}

headers = {
    'referer': 'https://news.naver.com/main/read.nhn?m_view=1&includeAllCount=true&mode=LS2D&mid=shm&sid1=101&sid2=259&oid=015&aid=0004352610'
}

comment_resp = requests.get(comment_base_url, params=params, headers=headers)

# 확인
comment_text = comment_resp.text
comment_text = comment_text[10:-2] # comment_text : 문자열.
comment_resp_dict = json.loads(comment_text) # json 데이터를 파이썬의 dictionary 형태로 바꿔 준다.
comment_list = comment_resp_dict['result']['commentList']
print(f"Total Comments: {len(comment_list)}")
```

그 사람의 댓글 히스토리를 받아오기 위해서는 다음과 같이 네트워크를 분석한다.

![comcom](./images/web_26.png)
![history](./images/web_27.png)